In [18]:
from fishsense_api_sdk.client import Client
from label_studio_sdk.client import LabelStudio
from pathlib import Path
from typing import List
from fishsense_api_sdk.models.species_label import SpeciesLabel
from tqdm.asyncio import tqdm_asyncio
from tqdm.notebook import tqdm
from fishsense_api_sdk.models.dive_slate_label import DiveSlateLabel
from fishsense_data_processing_workflow_worker.config import settings

In [19]:
PROJECT_ID = 66
DIVE_ID = 383

In [20]:
label_studio_api_key = Path("..") / ".label_studio_api_key"

label_studio_api_key.exists()

True

In [21]:
ls = LabelStudio(base_url=f"https://labeler.e4e.ucsd.edu", api_key=label_studio_api_key.read_text().strip())

In [22]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    dives = await fs.dives.get_canonical()
    dives = [dive for dive in dives if dive.priority == "HIGH" and dive.id == DIVE_ID]

len(dives), dives[0]

(1,
 Dive(id=383, name='062624_FiveSeas_FSL03', path='2025-02-10 REEF Data Dump SMILE 6/CubaTrip2024_FSL03/062624_FiveSeas_FSL03', dive_datetime=datetime.datetime(2024, 6, 26, 7, 52, 45, tzinfo=TzInfo(0)), priority=<Priority.HIGH: 'HIGH'>, flip_dive_slate=None, camera_id=3, dive_slate_id=10))

In [23]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    species_labels: List[SpeciesLabel] = await tqdm_asyncio.gather(*[fs.labels.get_species_labels(dive.id) for dive in dives])
    species_labels = [label for sublist in species_labels for label in sublist]

len(species_labels), species_labels[0]

100%|██████████| 1/1 [00:00<00:00,  2.30it/s]


(40,
 SpeciesLabel(id=1179, label_studio_task_id=224623, label_studio_project_id=70, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/708d385dc06f8fd60c46683478e77b2b', updated_at=datetime.datetime(2025, 12, 9, 1, 2, 5, 406207, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=None, laser_x=2043.1891467172798, laser_y=1355.6551899834365, laser_label='Red Laser', content_of_image='Slate, Laser on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 134871, 'result': [{'id': 'X-QEwfPU-K', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser on slate']]}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'species'}, {'id': 'w_p1HuzBVT', 'type': 'keypointlabels', 'value': {'x': 50.901573161865464, 'y': 44.94877950873463, 'width': 0.2762390456930156, 'keypointlabels': ['Red Laser']}, 'origin': 'manual', 'to_name': 'image',

In [24]:
slate_labels = [label for label in species_labels if label.content_of_image == 'Slate, Laser on slate']

len(slate_labels), slate_labels[0]

(9,
 SpeciesLabel(id=1179, label_studio_task_id=224623, label_studio_project_id=70, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/groups_jpeg/708d385dc06f8fd60c46683478e77b2b', updated_at=datetime.datetime(2025, 12, 9, 1, 2, 5, 406207, tzinfo=TzInfo(0)), completed=True, grouping=None, top_three_photos_of_group=None, slate_upside_down=None, laser_x=2043.1891467172798, laser_y=1355.6551899834365, laser_label='Red Laser', content_of_image='Slate, Laser on slate', fish_measurable_category=None, fish_angle_category=None, fish_curved_category=None, label_studio_json={'annotations': [{'id': 134871, 'result': [{'id': 'X-QEwfPU-K', 'type': 'taxonomy', 'value': {'taxonomy': [['Slate', 'Laser on slate']]}, 'origin': 'manual', 'to_name': 'image', 'from_name': 'species'}, {'id': 'w_p1HuzBVT', 'type': 'keypointlabels', 'value': {'x': 50.901573161865464, 'y': 44.94877950873463, 'width': 0.2762390456930156, 'keypointlabels': ['Red Laser']}, 'origin': 'manual', 'to_name': 'image', 

In [25]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    slate_images = await tqdm_asyncio.gather(*[fs.images.get(image_id=label.image_id) for label in slate_labels])

len(slate_images), slate_images[0]

100%|██████████| 9/9 [00:00<00:00, 32.79it/s]


(9,
 Image(id=111909, path='2025-02-10 REEF Data Dump SMILE 6/CubaTrip2024_FSL03/062624_FiveSeas_FSL03/P6260560.ORF', taken_datetime=datetime.datetime(2024, 6, 26, 6, 53, 31, tzinfo=TzInfo(0)), checksum='708d385dc06f8fd60c46683478e77b2b', is_canonical=True, dive_id=383, camera_id=3))

In [26]:
tasks = []

for image in tqdm(slate_images):
    tasks.append({
        "data": {
            "image": f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/{image.checksum}"
        },
        "predictions": [],
        "annotations": []
    })

tasks

  0%|          | 0/9 [00:00<?, ?it/s]

[{'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/708d385dc06f8fd60c46683478e77b2b'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/4711763b48368af835dc397c61411dab'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/a16c10d11e3ea788f966800c77af1501'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/d8c968f9f1582fc6218064d57196f6f0'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/c7d43584ed2aeee8b2eaa0aaa58a29d8'},
  'predictions': [],
  'annotations': []},
 {'data': {'image': 'https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/97bd3683b5a23ab2eda345ddbb73883b'},
  'predicti

In [27]:
imported_tasks = ls.projects.import_tasks(PROJECT_ID, request=tasks, return_task_ids=True)

imported_tasks

ImportTasksProjectsResponse(annotation_count=0, could_be_tasks_list=False, data_columns=[], duration=0.08802223205566406, file_upload_ids=[], found_formats=[], predictions_count=None, task_count=9, prediction_count=0, task_ids=[224784, 224785, 224786, 224787, 224788, 224789, 224790, 224791, 224792])

In [28]:
dive_slate_labels = [DiveSlateLabel(id=None, image_id=image.id, label_studio_task_id=task_id, label_studio_project_id=PROJECT_ID, image_url=f"https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/{image.checksum}", updated_at=None, completed=False, label_studio_json={}, user_id=None, reference_points=None, slate_rectangle=None, upside_down=None, skipped_points=None) for task_id, image in zip(imported_tasks.task_ids, slate_images)]

len(dive_slate_labels), dive_slate_labels[0]

(9,
 DiveSlateLabel(id=None, label_studio_task_id=224784, label_studio_project_id=66, image_url='https://orchestrator.fishsense.e4e.ucsd.edu/api/v1/data/dive_slate_jpgs/708d385dc06f8fd60c46683478e77b2b', upside_down=None, reference_points=None, slate_rectangle=None, skipped_points=None, updated_at=None, completed=False, label_studio_json={}, image_id=111909, user_id=None))

In [29]:
async with Client(settings.fishsense_api.url, settings.fishsense_api.username, settings.fishsense_api.password) as fs:
    for dive_slate_label in tqdm(dive_slate_labels):
        await fs.labels.put_dive_slate_label(dive_slate_label.image_id, dive_slate_label)

  0%|          | 0/9 [00:00<?, ?it/s]